# 20.基于模型的策略优化 (model-based policy optimization，MBPO）
- **Dyna-Q** 算法中的模型只存储之前遇到的数据，只适用于 **表格型环境**；
- 在 **连续型** 状态和动作的环境中，若继续利用 **Dyna** 的思想，可以像 **PETS** 算法一样学习一个用 **神经网络环境模型**，然后在任意状态和动作下用 **环境模型** 来生成一些虚拟数据来帮助进行策略的学习；
- 如此，对 **真实环境** 中样本的需求量会减少，通常会比 **无模型** 的强化学习方法具有更高的采样效率 —— **MBPO** 算法。

## 20.1 MBPO 原理
> **加州大学伯克利分校 (UC Berkeley)** 的研究员在 2019 年的 NeurIPS 会议中提出 **MBPO** 算法 [When to Trust Your Model: Model-Based Policy Optimization](https://arxiv.org/abs/1906.08253) 

> 研究证明，在多个 **MuJoCo** 连续控制任务上，**MBPO** 的性能相比 **基于模型** 的方法更优，甚至在样本效率上超过了一些 **无模型算法**（如 **SAC**）
> **MBPO** 被认为是 **基于模型的强化学习领域中的里程碑式算法**，成为深度强化学习中最重要的**基于模型**的强化学习算法之一，不少之后的工作都是在此基础上进行的

> **MBPO** 算法基于 **两个关键** 的观察：
- 随着环境模型的 **推演步数** 变长，模型累积的 **复合误差** 会快速增加，使得环境模型得出的结果变得很 **不可靠**
- **推演步数** 必须要 **权衡**  *推演步数长，复合误差大* 的 **负面作用** 与 *推演步数长，训练策略更优* 的 **正面作用**
> 在这两个观察的基础之上提出了一种 **分支推演（branched rollout）**  的思想：*只使用环境模型 **从之前访问过的真实状态开始** 进行 **较短步数** 的推演，**而非**从初始状态开始进行**完整的推演**，**避免**长时间依赖预测，**减弱**模型误差累积的影响：*
![示意图](Illustrations/分支推演示意图.png)
- **分支推演的长度$k$** 是 **平衡** 样本效率和策略性能的 **重要超参数**

### 真实环境下策略性能提升的单调性保障
> 基于模型的方法往往是在 **环境模型** 中提升策略的性能，但这并不能保证在 **真实环境** 中策略性能也有所提升，对此需要保证：

$$\eta[\pi]\geq\hat{\eta}[\pi]-\left[\frac{2\gamma r_{\max}(\epsilon_m+2\epsilon_\pi)}{(1-\gamma)^2}+\frac{4r_{\max}\epsilon_\pi}{(1-\gamma)}\right]$$
$$C=\left[\frac{2\gamma r_{\max}(\epsilon_m+2\epsilon_\pi)}{(1-\gamma)^2}+\frac{4r_{\max}\epsilon_\pi}{(1-\gamma)}\right]$$

- $\eta[\pi]$表示策略在 **真实环境** 中的 **期望回报**
- $\hat{\eta}[\pi]$表示策略在 **模型环境** 中的 **期望回报**
- $\epsilon_m$: **模型误差**，指模型在预测环境动态时的误差度量，代表着 **模型预测** 与 **真实环境状态转移**之间的差异（单步误差）
- $\epsilon_\pi$: **策略误差**，指策略执行时的误差度量，刻画了在同一策略$\pi$下，**模型预测的状态轨迹** 与 **真实轨迹** 之间的差异（长期误差），即 **复合误差** 

> 误差项的构造：
- $\frac{2\gamma r_{\max}(\epsilon_m + 2\epsilon_\pi)}{(1-\gamma)^2}$ 捕捉 **“模型不准 + 策略偏移”** 引起的长期累积误差
- $\frac{4r_{\max} \epsilon_\pi}{(1-\gamma)}$ 捕捉 **“即使模型准，但分布不同”** 造成的性能差
- 具体的推导过程见原论文附录 **Appendix A**

> 这意味着如果 **模型环境** 策略性能的提升可以超过 **performance bound（性能界/性能下界）$C$** ，理论上就可以在 **真实环境** 中取得策略性能的提升

### 模型推演长度理论选取指导

## 20.2 MBPO 代码实践

1. **MBPO** 算法与 **Dyna-Q** 算法十分类似。但 **Dyna-Q** 采用的**无模型强化学习部分**是 **Q-learning**，而 **MBPO** 是基于 **SAC**
2. **MBPO** 算法关于 **环境模型的构建** 和 **PETS** 算法中一致，都使用 **模型集成** 的方式
3. **MBPO** 算法使用了 **模型生成的数据** 和 **真实环境的数据** 混合训练策略，提升了 **样本效率** 和 **稳定性**